# Таблицы в PanDas
В этом лонгриде рассмотрим, что такое pandas-таблицы. Как их создавать и применять в работе

## Создание и устройство pandas-таблицы
Pandas-таблица похожа на двумерный numpy-массив с той разницей, что столбцы обычно (но не обязательно) имеют имена, по которым к ним можно обращаться (как словари).

Каждый столбец обычно хранит данные о каком-то признаке (температура, оценка, цена и т.д.).

Каждая строка обычно хранит данные об этих признаках, соответствующие индексу строки (время, имя, дата и т.д.).

Pandas можно назвать Excel'ем в Python. В своей практике вы наверняка будете работать с данными из Excel-документов именно с помощью Pandas.

Таблицы хранятся в объектах основного класса в Pandas - `DataFrame`.

![title](imgs/pandas_row_col.png)

Столбцами DataFrame'ов являются объекты второго по популярности класса в Pandas - `Series`.

![title](imgs/pandas_df_series.png)

Как и в случае numpy-массивов, Pandas-таблицы можно создать как с помощью специальных методов "с нуля", так и преобразовав в них другие структуры данных

In [1]:
# pd - распространённое сокращение при импорте библиотеки pandas
import pandas as pd
import numpy as np

Создадим столбец Series.

Ниже видно, что столбец, как и numpy-массив, знает о типе содержащихся в нём элементов, а элементы имеют индексы. Пока только числовые

In [2]:
pd.Series([5, 4, 10, 2])

0     5
1     4
2    10
3     2
dtype: int64

In [3]:
type(pd.Series([5, 4, 10, 2]))

pandas.core.series.Series

In [4]:
# к ним применимы срезы и многие операции, применимые к numpy-массивам
s = pd.Series([5, 4, 10, 2])
s[1:3]

1     4
2    10
dtype: int64

Создадим таблицу DataFrame с оценками трёх студентов.

Заметьте, что одно из значений задано как `NaN` - Not a Number. Это аналог `None` в Pandas, указыващий на пропущенные данные (фактически - на то, что ячейка есть, но она пустая)

In [5]:
# заметьте, Jupyter или Google Colab 
# автоматически выводят DataFrame в "красивом" виде
df = pd.DataFrame(
    [["Alex", 4, "male"],
    ["Svetlana", 5, "female"],
    ["Ivan", 3, "male"],
    ["Elena", np.nan, "female"]],
    columns=["student", "grade", "sex"])
df

,student,grade,sex
0,Alex,4.0,male
1,Svetlana,5.0,female
2,Ivan,3.0,male
3,Elena,NaN,female


Обратившись к конкретному столбцу (синтаксис похож на работу со словарём), можно получить столбец `Series`

In [6]:
# pandas постарается определить тип столбца
# если не получится - присвоит "абстрактный" тип object
df["student"]

0        Alex
1    Svetlana
2        Ivan
3       Elena
Name: student, dtype: object

In [7]:
df["grade"]

0    4.0
1    5.0
2    3.0
3    NaN
Name: grade, dtype: float64

Обратившись к нескольким столбцам, можно получить аналог срезов, только для таблиц.

Заметьте, что в случае нескольких столбцов требуется поставить 2 квадратные скобки

In [8]:
df[["student", "sex"]]

,student,sex
0,Alex,male
1,Svetlana,female
2,Ivan,male
3,Elena,female


In [9]:
# элементы любого столбца можно сделать индексами соответствующих строк
df.set_index("student", inplace=True)
df

,grade,sex
student,,
Alex,4.0,male
Svetlana,5.0,female
Ivan,3.0,male
Elena,NaN,female


In [10]:
# новый столбец можно добавить вручную
# как это с новыми ключами для словарей
df['birthdate'] = ['1970-01-12', '1972-05-12', '1989-01-14', '1971-12-02']
df

,grade,sex,birthdate
student,,,
Alex,4.0,male,1970-01-12
Svetlana,5.0,female,1972-05-12
Ivan,3.0,male,1989-01-14
Elena,NaN,female,1971-12-02


In [11]:
# выбрать нужную строку по её индексу можно с помощью метода loc
df.loc["Alex"]

grade               4.0
sex                male
birthdate    1970-01-12
Name: Alex, dtype: object

In [12]:
# а по номеру строки (индексу) - с помощью метода iloc
df.iloc[0]

grade               4.0
sex                male
birthdate    1970-01-12
Name: Alex, dtype: object

## Манипуляция со строками и столбцами
PanDas позволяет удобно преобразовывать элементы в строках, а особенно - в столбцах.

Например, метод `astype()` позволяет изменить тип данных, который будет приписываться элементам столбца. Это похоже на тип ячейки из Excel.

Тип `category` в блоке ниже используется, когда мы ожидаем, что переменная может принимать только ограниченное количество значений

In [13]:
# например, пол (не гендер) - муж/жен.
# category нужен при визуализации данных и работе с машинным обучением

df["sex"].astype("category")

student
Alex          male
Svetlana    female
Ivan          male
Elena       female
Name: sex, dtype: category
Categories (2, object): ['female', 'male']

Для более сложных типов данных (например, для дат) существуют особые названия. Подробнее о типах данных в PanDas можно почитать, например, [здесь](https://pbpython.com/pandas_dtypes.html)

In [14]:
df["birthdate"].astype("datetime64[ns]")

student
Alex       1970-01-12
Svetlana   1972-05-12
Ivan       1989-01-14
Elena      1971-12-02
Name: birthdate, dtype: datetime64[ns]

Большинство методов в Pandas не изменяют исходный `DataFrame`, а создают копии таблиц и столбцов.

Чтобы внести изменения в таблицу, нужно явно присвоить старому столбцу новый изменённый

In [15]:
df["birthdate"] = df["birthdate"].astype("datetime64[ns]")

Теперь можно "отфильтровывать" даты "по-умному", т.к. тип `datetime64` поддерживает арифметические операции с датами

In [16]:
# оставим только тех студентов, которые родились после 72-ого года
df[df['birthdate'] > '1972-01-01']

,grade,sex,birthdate
student,,,
Svetlana,5.0,female,1972-05-12
Ivan,3.0,male,1989-01-14


In [17]:
# условий может быть несколько
# не забудьте в таком случае выделить их скобками
# и разделить логическими знаками & или |
df[(df['birthdate'] > '1972-01-01') & (df['grade'] > 4)]

,grade,sex,birthdate
student,,,
Svetlana,5.0,female,1972-05-12


In [18]:
# теперь можно отсортировать студентов, например, по дате рождения
df.sort_values("birthdate", ascending=True)

,grade,sex,birthdate
student,,,
Alex,4.0,male,1970-01-12
Elena,NaN,female,1971-12-02
Svetlana,5.0,female,1972-05-12
Ivan,3.0,male,1989-01-14


## Применение функции к столбцу

Pandas позволяет применять функции ко всем элементам выбранного столбца с помощью метода `apply` по аналогии с `map` для списков или `vectorize` для numpy-массивов.

Ниже с помощью него вычисляется и сохраняется в отдельный столбец возраст каждого ученика


In [19]:
# библиотека datime позволяет удобно работать с датами
from datetime import datetime
import numpy as np

In [20]:
# создадим переменную с датой сегодняшнего дня
today = datetime.now()
today

datetime.datetime(2022, 11, 1, 23, 20, 41, 154998)

In [21]:
# и переменную, равную 365 дням
days_365 = np.timedelta64(365, 'D')
print(days_365)

365 days


Вычтем из сегодняшней даты дату рождения каждого ученика и поделим это на 365 дней

In [22]:
# в apply можно подставить как существующую функцию,
# так и lambda-функцию
ages = df['birthdate'].apply(lambda birthday: int((today - birthday)/days_365))
ages

student
Alex        52
Svetlana    50
Ivan        33
Elena       50
Name: birthdate, dtype: int64

Новый столбец можно поместить в таблицу по аналогии с созданием нового ключа и значения в словаре

In [23]:
df["age"] = ages
df

,grade,sex,birthdate,age
student,,,,
Alex,4.0,male,1970-01-12,52
Svetlana,5.0,female,1972-05-12,50
Ivan,3.0,male,1989-01-14,33
Elena,NaN,female,1971-12-02,50


In [24]:
# метод fillna позволяет заполнить все пустые ячейки конкретным значением
df.fillna(2)

,grade,sex,birthdate,age
student,,,,
Alex,4.0,male,1970-01-12,52
Svetlana,5.0,female,1972-05-12,50
Ivan,3.0,male,1989-01-14,33
Elena,2.0,female,1971-12-02,50


In [25]:
# строки могут иметь и одинаковые значения
df2 = df.append(df.iloc[3])
df2

C:\Users\user\AppData\Local\Temp\ipykernel_9240\449360623.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df.append(df.iloc[3])


,grade,sex,birthdate,age
student,,,,
Alex,4.0,male,1970-01-12,52
Svetlana,5.0,female,1972-05-12,50
Ivan,3.0,male,1989-01-14,33
Elena,NaN,female,1971-12-02,50
Elena,NaN,female,1971-12-02,50


In [26]:
# метод drop_duplicates() позволяет удалить повторяющиеся строки
df2.drop_duplicates()

,grade,sex,birthdate,age
student,,,,
Alex,4.0,male,1970-01-12,52
Svetlana,5.0,female,1972-05-12,50
Ivan,3.0,male,1989-01-14,33
Elena,NaN,female,1971-12-02,50


In [27]:
# причём можно удалить даже строки, в которых повторяются элементы в конкретных столбцах или группах столбцов
df2.drop_duplicates("sex")

,grade,sex,birthdate,age
student,,,,
Alex,4.0,male,1970-01-12,52
Svetlana,5.0,female,1972-05-12,50


## Работа с таблицами
Обычно вы будете не создавать таблицы с нуля, а будете импортировать данные из файлов или онлайн-репозиториев.

Методы для импорта данных из различных источников в `DataFrame` (например, из .csv или Excel-таблиц) подробно приведены [здесь](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

В качестве примера рассмотрим популярный набор данных `Titanic`, который можно скачать с помощью библиотеки для построения графиков `Seaborn`

In [28]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
type(titanic)

pandas.core.frame.DataFrame

In [29]:
# данных много (и строк, и столбцов)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [30]:
# поэтому можно посмотреть, например, на первые 5 строк с помощью метода head()
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [31]:
# или получить основную информацию о столбцах таблицы с помощью метода info()
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [32]:
# или выборку из n случайных строк с помощью метода sample
titanic.sample(n=10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
705,0,2,male,39.0,0,0,26.0000,S,Second,man,True,NaN,Southampton,no,True
763,1,1,female,36.0,1,2,120.0000,S,First,woman,False,B,Southampton,yes,False
339,0,1,male,45.0,0,0,35.5000,S,First,man,True,NaN,Southampton,no,True
451,0,3,male,NaN,1,0,19.9667,S,Third,man,True,NaN,Southampton,no,False
691,1,3,female,4.0,0,1,13.4167,C,Third,child,False,NaN,Cherbourg,yes,False
144,0,2,male,18.0,0,0,11.5000,S,Second,man,True,NaN,Southampton,no,True
594,0,2,male,37.0,1,0,26.0000,S,Second,man,True,NaN,Southampton,no,False
342,0,2,male,28.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
338,1,3,male,45.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,yes,True
884,0,3,male,25.0,0,0,7.0500,S,Third,man,True,NaN,Southampton,no,True


Для работы с табличными данными потребуется ещё множество методов. Нет смысла приводить их все в этом лонгриде.

Поэтому при работе с pandas советую держать под рукой шпаргалку. Например, [такую](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf).

А методы для удобной визуализации этих данных приведены в следующей лекции и лонгриде